## Tax calculator

In [ ]:
#Employee class
class Employee:
    #constructor of employee class
    def __init__(self, StaffID, LastName, FirstName, RegHours, HourlyRate, OTMultiple, TaxCredit, StandardBand):
        self.StaffID = StaffID
        self.LastName = LastName
        self.FirstName = FirstName
        self.RegHours = RegHours
        self.HourlyRate = HourlyRate
        self.OTMultiple = OTMultiple
        self.TaxCredit = TaxCredit
        self.StandardBand = StandardBand
        
    #Compute payment takes hours worked and date and returns the json
    def computePayment(self, HoursWorked, date):
        return o